In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('HPL.db')

In [2]:
def extract(filename, data):
    with open(filename, 'r') as file:
        lines = file.readlines()

    jump_cnt = 0

    # 遍历每一行并解析数据
    i = -1
    while i+1 < len(lines):
        i += 1
        line = lines[i]
        if "============" in line:
            if jump_cnt<2: # 跳过前两个=====
                jump_cnt += 1
                continue
            else: 
                if lines[i+2].split()[0] == 'Finished' or lines[i+1].split()[1] == 'BAD': 
                    i = len(lines)
                    break
                line = lines[i+3]
                parts = line.split()
                data.append(parts)
    return data

In [10]:
import os

directory_path = "../result/4/"  # 将此路径替换为您要搜索的目录的路径
file_names = [file for file in os.listdir(directory_path) if file.endswith(".out")]
data = []

for filename in file_names:
    print(filename)
    if 'fail' in filename: continue
    extract(directory_path + filename, data)

# 数据类型转换
columns_to_convert = range(1, 7)
data = [
    [float(value) if index in columns_to_convert else value for index, value in enumerate(row)]
    for row in data
]

# 加核数
data = [[4] + row for row in data]
df = pd.DataFrame(data, columns=['cores', 'T/V', 'N', 'NB', 'P', 'Q', 'Time', 'Gflops'])
df

N_12_cut.out
N_4_finish.out
N+_54_finish.out
NB_16_finish.out
NB_19_finish.out
NB_4_cut.out
NB_9_finish.out


,cores,T/V,N,NB,P,Q,Time,Gflops
0,4,WR00R2L2,40704.0,192.0,2.0,2.0,143.75,312.77
1,4,WR00R2L2,45696.0,192.0,2.0,2.0,200.89,316.66
2,4,WR00R2L2,49920.0,192.0,2.0,2.0,279.16,297.10
3,4,WR00R2L2,54144.0,192.0,2.0,2.0,349.04,303.18
4,4,WR00R2L2,57600.0,192.0,2.0,2.0,389.93,326.74
...,...,...,...,...,...,...,...,...
113,4,WR00R2L2,89088.0,368.0,2.0,2.0,1806.92,260.88
114,4,WR00R2L2,89088.0,376.0,2.0,2.0,1808.94,260.59
115,4,WR00R2L2,89088.0,384.0,2.0,2.0,2199.85,214.28
116,4,WR00R2L2,89088.0,392.0,2.0,2.0,2174.01,216.83


In [12]:
# 存储到数据库
df.to_sql('table1', conn, if_exists='replace', index=False)

118

In [13]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM table1 ORDER BY Gflops DESC LIMIT 1")

results = cursor.fetchall()
results

[(4, 'WR00R2L2', 89088.0, 328.0, 2.0, 2.0, 1294.77, 364.07)]

In [4]:
import os

directory_path = "../result/24/"  # 将此路径替换为您要搜索的目录的路径
file_names = [file for file in os.listdir(directory_path) if file.endswith(".out")]
data_24 = []

for filename in file_names:
    print(filename)
    if 'fail' in filename: continue
    extract(directory_path + filename, data_24)

# 数据类型转换
columns_to_convert = range(1, 7)
data_24 = [
    [float(value) if index in columns_to_convert else value for index, value in enumerate(row)]
    for row in data_24
]

# 加核数
data_24 = [[24] + row for row in data_24]
df = pd.DataFrame(data_24, columns=['cores', 'T/V', 'N', 'NB', 'P', 'Q', 'Time', 'Gflops'])
df

N&NB_51_cut.out
N&NB_PQ_74_cut.out
PQ_38_cut.out
other_9_finish.out
PQ+_20_cut.out
NB_10_finish.out
NB_16_finish.out


,cores,T/V,N,NB,P,Q,Time,Gflops
0,24,WR00R2L2,20352.0,312.0,4.0,6.0,7.08,794.34
1,24,WR00R2L2,20352.0,320.0,4.0,6.0,7.07,794.91
2,24,WR00R2L2,20352.0,328.0,4.0,6.0,6.55,858.64
3,24,WR00R2L2,28800.0,312.0,4.0,6.0,15.97,997.25
4,24,WR00R2L2,28800.0,320.0,4.0,6.0,16.26,979.46
...,...,...,...,...,...,...,...,...
213,24,WR00R2R2,79104.0,272.0,4.0,6.0,251.91,1310.00
214,24,WR00R2R2,79104.0,280.0,4.0,6.0,250.38,1318.00
215,24,WR00R2R2,79104.0,288.0,4.0,6.0,250.27,1318.60
216,24,WR00R2R2,79104.0,296.0,4.0,6.0,249.47,1322.80


In [5]:
df.to_sql("table1", conn, if_exists="append", index=False)

218

In [6]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM table1 ORDER BY Gflops DESC LIMIT 1")

results = cursor.fetchall()
results

[(24, 'WR00R2R2', 79104.0, 304.0, 4.0, 6.0, 248.36, 1328.7)]

In [ ]:
# 关闭游标和连接
cursor.close()

# 关闭数据库连接
conn.close()